In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("data/skylab_instagram_datathon_dataset.csv", sep=";")
df_original = df.copy(deep=True)
df.head()

,period,period_end_date,compset_group,compset,business_entity_doing_business_as_name,legal_entity_name,domicile_country_name,ultimate_parent_legal_entity_name,primary_exchange_name,calculation_type,followers,pictures,videos,comments,likes
0,Weekly,2019-12-07,Luxury & Premium & Mainstream,Soft Luxury,Versace,Capri Holdings,United States of America,Capri Holdings,New York Stock Exchange,Metric Value,22066189.0,97.0,13.0,16239.0,6112205.0
1,Weekly,2021-01-30,Luxury & Premium & Mainstream,Luxury & Premium & Mainstream,Versace,Capri Holdings,United States of America,Capri Holdings,New York Stock Exchange,Metric Value,24289963.0,54.0,2.0,13358.0,3189144.0
2,Weekly,2016-05-14,Luxury & Premium & Mainstream,Soft Luxury,Versace,Capri Holdings,United States of America,Capri Holdings,New York Stock Exchange,Metric Value,7137442.0,42.0,8.0,15894.0,2152955.0
3,Weekly,2019-04-20,Luxury & Premium & Mainstream,Luxury & Premium & Mainstream,Versace,Capri Holdings,United States of America,Capri Holdings,New York Stock Exchange,Metric Value,18901125.0,73.0,9.0,26762.0,5224916.0
4,Weekly,2023-03-25,Luxury & Premium & Mainstream,US Softlines Analyst Interest List,Versace,Capri Holdings,United States of America,Capri Holdings,New York Stock Exchange,Metric Value,30251374.0,54.0,10.0,25681.0,5760987.0


<h3>Removing columns "period", "calculation_type" <h3\>

In [3]:
#removing "period", "calculation_type" since the number of followers, likes,... are independent of these cathegories(Comment out after first run)
df = df_original.copy(deep=True) 
df.drop(columns=["period", "calculation_type", "domicile_country_name","primary_exchange_name", "compset"], inplace=True)
df.sort_values(by=["period_end_date"], inplace=True, ascending=False)
display(df.head())
display(df.shape)

,period_end_date,compset_group,business_entity_doing_business_as_name,legal_entity_name,ultimate_parent_legal_entity_name,followers,pictures,videos,comments,likes
533918,2023-09-16,Luxury & Premium & Mainstream,Moncler,Moncler,Moncler,5232719.0,6.0,4.0,361.0,63022.0
508933,2023-09-16,Luxury & Premium & Mainstream,IWC Schaffhausen,Richemont,Richemont,2426903.0,84.0,57.0,1094.0,178262.0
676605,2023-09-16,Apparel Retail,Pull&Bear,Inditex SA,Pontegadea Inversiones SL,7842273.0,18.0,21.0,777.0,170730.0
125631,2023-09-16,Luxury & Premium & Mainstream,Jaeger-LeCoultre,Richemont,Richemont,1346806.0,29.0,12.0,832.0,155136.0
96155,2023-09-16,Luxury & Premium & Mainstream,Armani,Armani,Armani,27257224.0,140.0,25.0,10208.0,782899.0


(704313, 10)

In [4]:
#Assumption: A brand will not be in different compset_groups over time!
df.drop_duplicates(inplace=True)
df.sort_values(by=["business_entity_doing_business_as_name", "period_end_date"], inplace=True, ascending=False)
display(df.shape)
display(df.head())

(325732, 10)

,period_end_date,compset_group,business_entity_doing_business_as_name,legal_entity_name,ultimate_parent_legal_entity_name,followers,pictures,videos,comments,likes
647704,2023-09-16,Toys & Collectibles,shopDisney,Disney,Disney,4239207.0,244.0,72.0,3908.0,330793.0
647623,2023-09-09,Toys & Collectibles,shopDisney,Disney,Disney,4233764.0,259.0,56.0,3700.0,335231.0
647803,2023-09-02,Toys & Collectibles,shopDisney,Disney,Disney,4228605.0,268.0,61.0,3920.0,341795.0
647742,2023-08-26,Toys & Collectibles,shopDisney,Disney,Disney,4224473.0,265.0,61.0,3494.0,316126.0
647869,2023-08-19,Toys & Collectibles,shopDisney,Disney,Disney,4221812.0,266.0,52.0,3099.0,266166.0


<h3> Using Timedelta instead of Absolute datetime and ordering by datetime <h3>

In [5]:
#taking timedelta from first available end date instead of absolute date (COMMENT OUT AFTER FIRST RUN)
df['period_end_date'] = pd.to_datetime(df['period_end_date'])
min_date = df['period_end_date'].min()
df['period_end_date'] = df['period_end_date'] - min_date
#order by period_end_date
df.sort_values(by="period_end_date", inplace=True)

<h3> where do we have missing values?<h3\>

In [6]:
for name in df.columns:
    if df[df[name].isnull()].shape[0] > 0:
        print("number of missing values for ",name, ": ",df[df[name].isnull()].shape[0])

number of missing values for  legal_entity_name :  27692
number of missing values for  ultimate_parent_legal_entity_name :  27692
number of missing values for  followers :  64137
number of missing values for  pictures :  3906
number of missing values for  videos :  9785
number of missing values for  comments :  4124
number of missing values for  likes :  3794


<h3>Investigation of Domicile Country Name<h3\>

In [7]:
#COMMENT OUT AFTER FIRST RUN
#nan_domicile = df[df['domicile_country_name'].isnull()]
#nan_domicile["business_entity_doing_business_as_name"].unique().size
#check if brand has non-null domicile for each brand
#for brand in nan_domicile["business_entity_doing_business_as_name"].unique():
    #known_brands = []
    #check if brand has non-null domicile
    #if df[df["business_entity_doing_business_as_name"] == brand]["domicile_country_name"].notnull().any():
        #known_brands.append(brand)
        #print(brand)
#the result is empty, so we can't get the domicile from another row
#remove domicile_country_name column
#df.drop(columns=["domicile_country_name"], inplace=True)

In [15]:
cathegories = ["followers", "likes", "comments", "videos", "pictures"]
for name in cathegories:
    print("number of missing values for ",name, ": ",df[df[name].isnull()].shape[0] + df[df[name] == 0].shape[0])
    print("number of zeros for ",name, ": ",df[df[name] == 0].shape[0])
    #ratio of zeros + nan to total
    print("ratio of zeros + nan to total for ",name, ": ",(df[df[name].isnull()].shape[0] + df[df[name] == 0].shape[0])/df.shape[0])
    


number of missing values for  followers :  64150
number of zeros for  followers :  13
ratio of zeros + nan to total for  followers :  0.1969410435572802
number of missing values for  likes :  9198
number of zeros for  likes :  5404
ratio of zeros + nan to total for  likes :  0.028237937936708705
number of missing values for  comments :  10280
number of zeros for  comments :  6156
ratio of zeros + nan to total for  comments :  0.03155968710473641
number of missing values for  videos :  51571
number of zeros for  videos :  41786
ratio of zeros + nan to total for  videos :  0.15832340697260325
number of missing values for  pictures :  10934
number of zeros for  pictures :  7028
ratio of zeros + nan to total for  pictures :  0.033567472646224505


In [22]:
#fill nan with zeros
for name in cathegories:
    df[name].fillna(0, inplace=True)

/var/folders/xn/dh3j6gq15v5cw7ldfp8cpgrr0000gn/T/ipykernel_78346/996413486.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[name].fillna(0, inplace=True)


In [20]:
# brands_missing_values = pd.DataFrame(columns=["brand", "cathegory", "ratio"])

# for name in df["business_entity_doing_business_as_name"].unique():
#     for cathegory in cathegories:
#         nr_missing_values = df[(df["business_entity_doing_business_as_name"] == name) & (df[cathegory].isnull() | (df[cathegory] == 0))].shape[0]
#         ratio = nr_missing_values / df[df["business_entity_doing_business_as_name"] == name].shape[0]
#         if ratio > 0.3:
#             brands_missing_values.loc[len(brands_missing_values.index)] = [name, cathegory, ratio]
#             #print("brand: ",name, " cathegory: ",cathegory, " ratio: ",ratio)
#brands_missing_values.to_csv("data/brands_missing_values.csv")

In [9]:
#filling missing values with the mean of the brand 


# for name in df["business_entity_doing_business_as_name"].unique():
#     for cathegory in cathegories:
#         #if there are zeros or nan values for a brand, fill them with the mean of the brand
#         if df[(df["business_entity_doing_business_as_name"] == name) & (df[cathegory].isnull() | (df[cathegory] == 0))].shape[0] > 0:
#             mean = df[df["business_entity_doing_business_as_name"] == name][cathegory].mean()
#             df.loc[(df["business_entity_doing_business_as_name"] == name) & (df[cathegory].isnull() | (df[cathegory] == 0)), cathegory] = mean


/var/folders/xn/dh3j6gq15v5cw7ldfp8cpgrr0000gn/T/ipykernel_78346/2455941859.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  nan_followers[df["business_entity_doing_business_as_name"] == "Bottega Veneta"].size


430

In [11]:
#save the cleaned data
df.to_csv("data/cleaned_data.csv")